In [2]:
import pandas as pd
import numpy as np

In [21]:
df1 = pd.read_csv('wikipages_nlp2.csv', index_col=0)
df1.head(10)

,Name,Year,inTransWomen,inTransMen,inNB,inTrans,inWomen,inMen,inLGBT,inBisexual_people,...,empath_weapon,empath_children,empath_monster,empath_ocean,empath_giving,empath_contentment,empath_writing,empath_rural,empath_positive_emotion,empath_musical
0,Kjell Aamot,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.006897,0.000000,0.000000,0.000000
1,Olav Aaraas,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.006211,0.000000,0.000000,0.000000
2,Marc Aaronson,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.001025,0.000000,0.0,0.000000,0.000000,0.001025,0.000000
3,Ülle Aaskivi,1950,False,False,False,False,True,False,False,False,...,0.000000,0.002304,0.0,0.000000,0.002304,0.0,0.000000,0.000000,0.002304,0.000000
4,Oğuz Abadan,1950,False,False,False,False,False,False,False,False,...,0.000000,0.003942,0.0,0.000000,0.001314,0.0,0.000000,0.000000,0.000000,0.024967
5,Ahmet Abakay,1950,False,False,False,False,False,True,False,False,...,0.000000,0.015075,0.0,0.000000,0.000000,0.0,0.012563,0.001256,0.003769,0.000000
6,Martine Aballéa,1950,False,False,False,False,True,False,True,False,...,0.000000,0.001067,0.0,0.000000,0.000000,0.0,0.001067,0.000000,0.000000,0.003202
7,Gadzhi Abashilov,1950,False,False,False,False,False,False,False,False,...,0.001996,0.001996,0.0,0.000000,0.000000,0.0,0.003992,0.000000,0.001996,0.000000
8,Kamil Al-Abbasi,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.008333,0.000000,0.000000,0.000000
9,Aslan Abbasov,1950,False,False,False,False,False,False,False,False,...,0.000000,0.000000,0.0,0.000000,0.003953,0.0,0.035573,0.000000,0.000000,0.000000


In [33]:
df1=df1.drop(df1.columns[2:13],axis=1)#df1.columns[2:13]
df1.head()

,Name,Year,links,categories,QID,text,word_count,vader_compound,vader_neg,vader_neu,...,empath_weapon,empath_children,empath_monster,empath_ocean,empath_giving,empath_contentment,empath_writing,empath_rural,empath_positive_emotion,empath_musical
0,Kjell Aamot,1950,"['BI Norwegian Business School', 'Verdens Gang']","['Category:1950 births', 'Category:BI Norwegia...",Q11980824,Norwegian business executive Kjell Aamot Born ...,185,0.4019,0.000,0.979,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.006897,0.0,0.000000,0.000000
1,Olav Aaraas,1950,"['Fredrikstad', 'Maihaugen', 'Norwegian Museum...","['Category:1950 births', 'Category:Directors o...",Q3431302,Norwegian historian and museum director Olav A...,187,0.7003,0.000,0.951,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.006211,0.0,0.000000,0.000000
2,Marc Aaronson,1950,"['United States', 'Astronomer', 'Los Angeles, ...","['Category:1950 births', 'Category:1987 deaths...",Q2258893,American astronomer Marc Aaronson Born ( 1950-...,1098,0.9826,0.032,0.894,...,0.0,0.000000,0.0,0.001025,0.000000,0.0,0.000000,0.0,0.001025,0.000000
3,Ülle Aaskivi,1950,"['1990 Estonian Supreme Soviet election', 'Con...","['Category:1950 births', 'Category:2007 deaths...",Q16407530,Estonian politician Ülle Aaskivi Born Ülle Kii...,1039,0.9731,0.041,0.888,...,0.0,0.002304,0.0,0.000000,0.002304,0.0,0.000000,0.0,0.002304,0.000000
4,Oğuz Abadan,1950,"['Turkey', 'Ankara', 'Mandolin', 'Gazi Univers...","['Category:1950 births', 'Category:Living peop...",Q7116769,Turkish musician (born 1950) This biography of...,845,0.9305,0.007,0.961,...,0.0,0.003942,0.0,0.000000,0.001314,0.0,0.000000,0.0,0.000000,0.024967


In [29]:
#df1['vader_neg'].describe()

count    743219.000000
mean          0.022242
std           0.028391
min           0.000000
25%           0.000000
50%           0.014000
75%           0.031000
max           0.395000
Name: vader_neg, dtype: float64

In [19]:
df2 = pd.read_csv('wikidataSupportData.csv',keep_default_na=False)

In [46]:
df2.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref
0,Q66764,Marion Becker,female,,,,,
1,Q69588,Yuri N Maltsev,male,,,,,
2,Q70195,Klaus Regling,male,,,,,
3,Q74800,Hans-Joachim Zillmer,male,,,,,
4,Q75733,Gisbert Haefs,male,,,,,


In [65]:
df=df2[~df2.P21.str.contains('|'.join(['http://www.wikidata.org/']))]
df=df[~df.P91.str.contains('|'.join(['http://www.wikidata.org/']))]
df['P91'].unique()

array(['', 'lesbianism', 'non-heterosexuality', 'asexuality', 'gay',
       'homosexuality', 'heterosexuality', 'bisexuality', 'pansexuality',
       'hasbian', 'homoeroticism', 'sapiosexuality', 'male homosexuality',
       'demisexuality', 'homoromantic', 'sexual fluidity', 'aromanticism',
       'gray asexuality', 'bisexual', 'gay men'], dtype=object)

In [55]:
ng=pd.concat([df[df['P91']==''],df[df['P91']=='heterosexuality']])
g=df[~df.QID.isin(ng.QID)]

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref
0,Q66764,Marion Becker,female,,,,,
1,Q69588,Yuri N Maltsev,male,,,,,
2,Q70195,Klaus Regling,male,,,,,
3,Q74800,Hans-Joachim Zillmer,male,,,,,
4,Q75733,Gisbert Haefs,male,,,,,


In [59]:
ng.head()

In [60]:
g.head()

,QID,Name,P21,P21ref,P91,P91ref,P6553,P6553ref
5043,Q6794690,Max Dashu,female,,lesbianism,,,
5469,Q7280044,Radclyffe,female,,lesbianism,,,
9181,Q16543603,Carmen Muñoz,female,,lesbianism,,,
11473,Q21567781,Tamás Király,male,,non-heterosexuality,,,
11593,Q22004198,Beth Elliott,transgender female,,lesbianism,,,


In [73]:
cis=pd.concat([df[df['P21']==''],
             df[df['P21']=='female'],
             df[df['P21']=='cisgender male'],
             df[df['P21']=='cisgender female'],
             df[df['P21']=='male']])
t=df[~df.QID.isin(cis.QID)]

In [74]:
t['P21'].unique()

array(['transgender female', 'non-binary', 'genderqueer',
       'transgender male', 'intersex', 'kathoey', 'two-spirit',
       'genderfluid', 'transgender person', 'third gender', 'neutral sex',
       'travesti', 'bigender', 'female organism', 'pangender', 'agender',
       'queer', 'male organism', 'muxe', 'intersex person',
       'transmasculine', 'transfeminine', 'demiboy', 'neutrois',
       'assigned female at birth', 'nigger'], dtype=object)

In [75]:
cis['P21'].unique()

array(['', 'female', 'cisgender male', 'cisgender female', 'male'],
      dtype=object)

In [68]:
df['P21'].unique()

array(['female', 'male', 'transgender female', '', 'intersex',
       'non-binary', 'genderqueer', 'transgender male',
       'transgender person', 'kathoey', 'two-spirit', 'cisgender male',
       'genderfluid', 'third gender', 'neutral sex', 'transfeminine',
       'travesti', 'cisgender female', 'bigender', 'female organism',
       'pangender', 'agender', 'queer', 'male organism',
       'intersex person', 'muxe', 'transmasculine', 'demiboy', 'neutrois',
       'assigned female at birth', 'nigger'], dtype=object)